In [1]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"eronepius","key":"d1299d9c29ad703cb5b9a5e29f3dfb1e"}'}

In [3]:
!pip install -q kaggle

In [4]:
!ls

kaggle.json  sample_data


In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d rtatman/blog-authorship-corpus

 99% 288M/290M [00:02<00:00, 125MB/s]
100% 290M/290M [00:02<00:00, 118MB/s]


In [7]:
!ls

blog-authorship-corpus.zip  kaggle.json  sample_data


In [8]:
!unzip blog-authorship-corpus.zip -d blog-authorship-corpus

Archive:  blog-authorship-corpus.zip
  inflating: blog-authorship-corpus/blogtext.csv  


## Load the dataset

In [52]:
import pandas as pd

df = pd.read_csv('blog-authorship-corpus/blogtext.csv', nrows= 5000)

In [53]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [54]:
df.shape

(5000, 7)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   gender  5000 non-null   object
 2   age     5000 non-null   int64 
 3   topic   5000 non-null   object
 4   sign    5000 non-null   object
 5   date    5000 non-null   object
 6   text    5000 non-null   object
dtypes: int64(2), object(5)
memory usage: 273.6+ KB


- There is no column with null value

In [56]:
df1 = df[['gender','topic','sign','date','text']]

In [57]:
df1.head()

,gender,topic,sign,date,text
0,male,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,male,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,male,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,male,Student,Leo,"12,May,2004",testing!!! testing!!!
4,male,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


Preprocess rows of the “text” column<br>
a. Remove unwanted characters<br>
b. Convert text to lowercase<br>
c. Remove unwanted spaces<br>
d. Remove stopwords<br>

In [58]:
import re

try:
  df1 = df1.applymap(lambda s: s.lower())
except Exception as e:
  pass

In [59]:
try:
  df1 = df1.applymap(lambda s: re.sub('[^a-zA-Z0-9#+_]',' ',s))
except Exception as e:
  pass



In [60]:
try:
  df1 = df1.applymap(lambda s: re.sub(' +', ' ',s))
except Exception as e:
  pass



In [61]:
try:
  df1 = df1.applymap(lambda s: s.strip())
except Exception as e:
  pass

In [62]:
df1.head()

,gender,topic,sign,date,text
0,male,student,leo,14 may 2004,info has been found + 100 pages and 4 5 mb of ...
1,male,student,leo,13 may 2004,these are the team members drewes van der laag...
2,male,student,leo,12 may 2004,in het kader van kernfusie op aarde maak je ei...
3,male,student,leo,12 may 2004,testing testing
4,male,investmentbanking,aquarius,11 june 2004,thanks to yahoo s toolbar i can now capture th...


In [63]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
from nltk.corpus import stopwords

cachedStopWords = set(stopwords.words("english"))

#df1['text_no_stopwords'] = df1['text'].apply(lambda x: [item for item in x if item not in cachedStopWords])
df1['text_no_stopwords'] = df1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (cachedStopWords)]))


In [65]:
df1.head()

,gender,topic,sign,date,text,text_no_stopwords
0,male,student,leo,14 may 2004,info has been found + 100 pages and 4 5 mb of ...,info found + 100 pages 4 5 mb pdf files wait u...
1,male,student,leo,13 may 2004,these are the team members drewes van der laag...,team members drewes van der laag urllink mail ...
2,male,student,leo,12 may 2004,in het kader van kernfusie op aarde maak je ei...,het kader van kernfusie op aarde maak je eigen...
3,male,student,leo,12 may 2004,testing testing,testing testing
4,male,investmentbanking,aquarius,11 june 2004,thanks to yahoo s toolbar i can now capture th...,thanks yahoo toolbar capture urls popups means...


In [66]:
df1['age'] = df['age']

In [67]:
df1.head()

,gender,topic,sign,date,text,text_no_stopwords,age
0,male,student,leo,14 may 2004,info has been found + 100 pages and 4 5 mb of ...,info found + 100 pages 4 5 mb pdf files wait u...,15
1,male,student,leo,13 may 2004,these are the team members drewes van der laag...,team members drewes van der laag urllink mail ...,15
2,male,student,leo,12 may 2004,in het kader van kernfusie op aarde maak je ei...,het kader van kernfusie op aarde maak je eigen...,15
3,male,student,leo,12 may 2004,testing testing,testing testing,15
4,male,investmentbanking,aquarius,11 june 2004,thanks to yahoo s toolbar i can now capture th...,thanks yahoo toolbar capture urls popups means...,33


In [68]:
df1['labels'] = df1['gender'] + ' ' +  df1['age'].astype(str) + ' ' + df1['topic'] + ' ' + df1['sign']

In [69]:
df1.head()

,gender,topic,sign,date,text,text_no_stopwords,age,labels
0,male,student,leo,14 may 2004,info has been found + 100 pages and 4 5 mb of ...,info found + 100 pages 4 5 mb pdf files wait u...,15,male 15 student leo
1,male,student,leo,13 may 2004,these are the team members drewes van der laag...,team members drewes van der laag urllink mail ...,15,male 15 student leo
2,male,student,leo,12 may 2004,in het kader van kernfusie op aarde maak je ei...,het kader van kernfusie op aarde maak je eigen...,15,male 15 student leo
3,male,student,leo,12 may 2004,testing testing,testing testing,15,male 15 student leo
4,male,investmentbanking,aquarius,11 june 2004,thanks to yahoo s toolbar i can now capture th...,thanks yahoo toolbar capture urls popups means...,33,male 33 investmentbanking aquarius


a. Label columns to merge: “gender”, “age”, “topic”, “sign”

In [70]:
df1['labels'] = df1['labels'].str.split(" ")

In [71]:
df1.head()

,gender,topic,sign,date,text,text_no_stopwords,age,labels
0,male,student,leo,14 may 2004,info has been found + 100 pages and 4 5 mb of ...,info found + 100 pages 4 5 mb pdf files wait u...,15,"[male, 15, student, leo]"
1,male,student,leo,13 may 2004,these are the team members drewes van der laag...,team members drewes van der laag urllink mail ...,15,"[male, 15, student, leo]"
2,male,student,leo,12 may 2004,in het kader van kernfusie op aarde maak je ei...,het kader van kernfusie op aarde maak je eigen...,15,"[male, 15, student, leo]"
3,male,student,leo,12 may 2004,testing testing,testing testing,15,"[male, 15, student, leo]"
4,male,investmentbanking,aquarius,11 june 2004,thanks to yahoo s toolbar i can now capture th...,thanks yahoo toolbar capture urls popups means...,33,"[male, 33, investmentbanking, aquarius]"


b. After completing the previous step, there should be only two columns in your data
frame i.e. “text” and “labels” as shown in the below image

In [72]:
df2 = df1[['text_no_stopwords', 'labels']]

In [73]:
df2.head()

,text_no_stopwords,labels
0,info found + 100 pages 4 5 mb pdf files wait u...,"[male, 15, student, leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, student, leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, student, leo]"
3,testing testing,"[male, 15, student, leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, investmentbanking, aquarius]"


### Seperating features and target labels and creating training and test dataset

In [74]:
from sklearn.model_selection import train_test_split

X = df2['text_no_stopwords']
y = df2['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range = (1, 2), stop_words = 'english', min_df = 2)
X_train_p = vectorizer.fit_transform(X_train)

In [76]:
X_test_p = vectorizer.transform(X_test)

In [77]:
print(X_train_p)

  (0, 28560)	1
  (0, 11951)	1
  (1, 28145)	3
  (1, 28749)	1
  (1, 10850)	1
  (1, 24060)	1
  (1, 7235)	1
  (1, 3946)	1
  (1, 14472)	2
  (1, 16598)	1
  (1, 6300)	1
  (1, 28044)	1
  (1, 17608)	2
  (1, 28197)	2
  (1, 14865)	1
  (1, 15341)	1
  (1, 10808)	1
  (1, 28744)	1
  (1, 3655)	1
  (1, 13682)	1
  (1, 23315)	1
  (1, 24735)	2
  (1, 27917)	1
  (1, 28094)	1
  (1, 23388)	1
  :	:
  (3748, 9968)	1
  (3748, 3026)	1
  (3748, 20325)	1
  (3748, 4099)	1
  (3748, 28554)	1
  (3748, 17107)	1
  (3748, 13509)	3
  (3748, 10262)	1
  (3748, 3844)	1
  (3748, 11360)	1
  (3748, 23246)	1
  (3748, 20979)	1
  (3748, 11550)	1
  (3748, 17980)	1
  (3748, 26058)	1
  (3748, 5896)	1
  (3748, 21574)	1
  (3748, 17679)	1
  (3748, 166)	1
  (3748, 11361)	1
  (3748, 12922)	1
  (3748, 28154)	1
  (3748, 16174)	1
  (3749, 7017)	1
  (3749, 28461)	1


In [78]:
print(X_test_p)

  (0, 4929)	1
  (0, 7141)	1
  (0, 9715)	1
  (0, 11129)	1
  (1, 742)	1
  (1, 1822)	1
  (1, 2075)	1
  (1, 2568)	1
  (1, 2619)	1
  (1, 2955)	1
  (1, 3097)	1
  (1, 4656)	1
  (1, 5281)	2
  (1, 5293)	1
  (1, 5405)	2
  (1, 5624)	1
  (1, 5778)	1
  (1, 5977)	1
  (1, 7179)	1
  (1, 7502)	1
  (1, 7808)	1
  (1, 7836)	2
  (1, 7875)	1
  (1, 8228)	1
  (1, 9017)	1
  :	:
  (1249, 24519)	1
  (1249, 24991)	1
  (1249, 25038)	1
  (1249, 25625)	1
  (1249, 25723)	1
  (1249, 26134)	1
  (1249, 26489)	4
  (1249, 26506)	1
  (1249, 26563)	2
  (1249, 26787)	2
  (1249, 26804)	1
  (1249, 26831)	1
  (1249, 26844)	2
  (1249, 26868)	2
  (1249, 26876)	1
  (1249, 27290)	1
  (1249, 27334)	1
  (1249, 27336)	1
  (1249, 27536)	1
  (1249, 27602)	1
  (1249, 27917)	1
  (1249, 28032)	1
  (1249, 28648)	1
  (1249, 28774)	1
  (1249, 29297)	5


## Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label.

In [79]:
import warnings
warnings.filterwarnings('ignore')

In [80]:
dfT = df1[['gender', 'age', 'topic', 'sign']]
dfT['age'] = dfT['age'].astype('str')
keys=[] 
values=[] 

for i in range(dfT.shape[1]): # iterate through all the colummns        
    for j in range(dfT.iloc[:,i].value_counts().shape[0]): # iterate through all the rows of value_counts of that column
        keys.append(dfT.iloc[:,i].value_counts().index[j])         
        values.append(dfT.iloc[:,i].value_counts().iloc[j])
dictionary = dict(zip(keys,values))

In [81]:
import pprint

pprint.pprint(dictionary)

{'14': 170,
 '15': 339,
 '16': 67,
 '17': 331,
 '23': 137,
 '24': 353,
 '25': 268,
 '26': 96,
 '27': 86,
 '33': 101,
 '34': 540,
 '35': 2307,
 '36': 60,
 '37': 19,
 '39': 79,
 '41': 14,
 '42': 9,
 '44': 3,
 '45': 14,
 '46': 7,
 'accounting': 2,
 'aquarius': 329,
 'aries': 2483,
 'arts': 31,
 'automotive': 14,
 'banking': 16,
 'businessservices': 87,
 'cancer': 94,
 'capricorn': 84,
 'communications media': 61,
 'consulting': 16,
 'education': 118,
 'engineering': 119,
 'female': 1706,
 'gemini': 86,
 'indunk': 1381,
 'internet': 20,
 'investmentbanking': 70,
 'law': 3,
 'leo': 190,
 'libra': 414,
 'male': 3294,
 'museums libraries': 2,
 'non profit': 47,
 'pisces': 67,
 'religion': 4,
 'sagittarius': 704,
 'science': 33,
 'scorpio': 408,
 'sports recreation': 75,
 'student': 569,
 'taurus': 100,
 'technology': 2332,
 'virgo': 41}


## Transform the labels <br>
a. Convert your train and test labels using MultiLabelBinarizer

In [82]:
from sklearn.preprocessing import MultiLabelBinarizer

MLaBi = MultiLabelBinarizer(classes = sorted(dictionary.keys()))
y_train_MLaBi = MLaBi.fit_transform(y_train)
y_test_MLaBi = MLaBi.transform(y_test)

In [83]:
y_train_MLaBi[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [84]:
y_test_MLaBi[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

##  Choose a classifier 

In [85]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs')
classifier = OneVsRestClassifier(classifier)
classifier.fit(X_train_p, y_train_MLaBi)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [86]:
y_pred_train = classifier.predict(X_train_p)
y_pred_train

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [87]:
y_pred_test = classifier.predict(X_test_p)
y_pred_test

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

Fit the classifier, make predictions and get the accuracy (5 points)<br>
a. Print the following<br>
i. Accuracy score<br>
ii. F1 score<br>
iii. Average precision score<br>
iv. Average recall score<br>

In [88]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score

def print_scores(actual, predicted, averaging_type):
    print('\nAveraging Type : {}'.format(averaging_type))
    print('F1 score: {}'.format(f1_score(actual, predicted, average=averaging_type)))
    print('Average Precision Score: {}'.format(average_precision_score(actual, predicted, average=averaging_type)))
    print('Average Recall Score: {}'.format(recall_score(actual, predicted, average=averaging_type)))

In [89]:
print('--' * 20 + 'TRAIN SCORES'+ '--' * 20)
print('Accuracy score: {}'.format(accuracy_score(y_train_MLaBi, y_pred_train)))
print_scores(y_train_MLaBi, y_pred_train, 'micro')
y_val_true, val_pred = y_train_MLaBi.reshape((-1)), y_pred_train.reshape((-1))
print_scores(y_val_true, val_pred, 'macro')
print_scores(y_train_MLaBi, y_pred_train, 'weighted')

----------------------------------------TRAIN SCORES----------------------------------------
Accuracy score: 0.9554666666666667

Averaging Type : micro
F1 score: 0.9780942692438114
Average Precision Score: 0.9589014770891543
Average Recall Score: 0.9703942941730588

Averaging Type : macro
F1 score: 0.9881869711109472
Average Precision Score: 0.9589014770891543
Average Recall Score: 0.9846482177651765

Averaging Type : weighted
F1 score: 0.9777181616535846
Average Precision Score: 0.9605971546598773
Average Recall Score: 0.9703942941730588


In [90]:
print('--' * 20 + 'TRAIN SCORES'+ '--' * 20)
print('Accuracy score: {}'.format(accuracy_score(y_test_MLaBi, y_pred_test)))
print_scores(y_train_MLaBi, y_pred_train, 'micro')
y_val_true, val_pred = y_train_MLaBi.reshape((-1)), y_pred_train.reshape((-1))
print_scores(y_val_true, val_pred, 'macro')
print_scores(y_train_MLaBi, y_pred_train, 'weighted')

----------------------------------------TRAIN SCORES----------------------------------------
Accuracy score: 0.552

Averaging Type : micro
F1 score: 0.9780942692438114
Average Precision Score: 0.9589014770891543
Average Recall Score: 0.9703942941730588

Averaging Type : macro
F1 score: 0.9881869711109472
Average Precision Score: 0.9589014770891543
Average Recall Score: 0.9846482177651765

Averaging Type : weighted
F1 score: 0.9777181616535846
Average Precision Score: 0.9605971546598773
Average Recall Score: 0.9703942941730588


## Print true label and predicted label for any five examples 

In [108]:
five_data = y_test_MLaBi[10:15]
five_pred = y_pred_test[10:15]

In [109]:
five_data = MLaBi.inverse_transform(five_data)
print("True Label \n")
five_data

True Label 



[('35', 'aries', 'male', 'technology'),
 ('16', 'gemini', 'indunk', 'male'),
 ('35', 'aries', 'male', 'technology'),
 ('25', 'female', 'indunk', 'libra'),
 ('14', 'indunk', 'male', 'scorpio')]

In [110]:
five_pred = MLaBi.inverse_transform(five_pred)
print("Predicated Label \n")
five_pred

Predicated Label 



[('35', 'aries', 'male', 'technology'),
 ('female', 'indunk', 'scorpio'),
 ('35', 'aries', 'male', 'technology'),
 ('25', 'female', 'indunk', 'libra'),
 ('35', 'aries', 'male', 'technology')]